<a href="https://colab.research.google.com/github/ekmungi/ml_courses/blob/master/rnn_addition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch torchvision

In [0]:
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F


In [0]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, nlayers, output_dim):
        super(RNNModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.nlayers = nlayers
        
        self.rnn = nn.RNN(input_size=input_dim, hidden_size=hidden_dim,
                          num_layers=nlayers, batch_first=True, nonlinearity='tanh')
        
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        if torch.cuda.is_available():
            h0 = torch.zeros(self.nlayers, x.size(0), self.hidden_dim).cuda()
        else:
            h0 = torch.zeros(self.nlayers, x.size(0), self.hidden_dim)
            
            
        out, h = self.rnn(x, h0)
        
        out = self.fc(out)
        
        return torch.sigmoid(out)
            
        
        

In [0]:
int2binary = {}
binary_dim = 8
largest_number = pow(2,binary_dim)
binary = np.unpackbits(np.array([range(largest_number)],dtype=np.uint8).T,axis=1)
for i in range(largest_number):
    int2binary[i] = binary[i]

In [0]:
input_dim = 2
hidden_dim = 16
layer_dim = 1  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 1

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

In [0]:
if torch.cuda.is_available():
    model.cuda()

In [0]:
criterion = nn.MSELoss()

In [0]:
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [0]:
for j in range(1000000):
    a_int = np.random.randint(largest_number/2)
    a = int2binary[a_int]
    
    b_int = np.random.randint(largest_number/2)
    b = int2binary[b_int]
    
    c_int = a_int + b_int
    c = int2binary[c_int]
    
    d = np.zeros_like(c)
    
    
    
    X = []
    y = []
    
    for position in range(binary_dim):
        X.extend([[a[binary_dim - position - 1],b[binary_dim - position - 1]]])
        y.extend([c[binary_dim - position - 1]])
       
        
    X = torch.from_numpy(np.expand_dims(np.array(X), axis=0)).float()
    y = torch.from_numpy(np.squeeze(np.array(y))).float()
       
    if torch.cuda.is_available():
        X = X.cuda()
        y = y.cuda()    
    
    optimizer.zero_grad()
    
    y_pred = model(X)
        
    loss = criterion(torch.squeeze(torch.squeeze(y_pred)), y)
    
    loss.backward()
    
    optimizer.step()
    
    if(j % 5000 == 0):
        with torch.no_grad():
            out = 0
            for index in range(binary_dim):
                 out += torch.squeeze(torch.squeeze(y_pred))[index]*pow(2,index)
            print('Pred: {0}, True: {1}'.format(int(np.round(out.cpu().numpy())), c_int))
                 
#             print("Error: {0}".format(loss.detach()))
#             print("Pred: {0}".format(torch.squeeze(torch.squeeze(y_pred)).cpu().detach().numpy()))
#             print("True: {0}".format(y.cpu().detach().numpy()))
#             print("-----------------------------")
            
        
        

Pred: 115, True: 195
Pred: 129, True: 123
Pred: 176, True: 178
Pred: 217, True: 222
Pred: 19, True: 16
Pred: 96, True: 96
Pred: 102, True: 102


In [65]:
rnn = nn.RNN(input_size=input_dim, hidden_size=hidden_dim, 
             num_layers=layer_dim, batch_first=True, nonlinearity='tanh')
fc = nn.Linear(hidden_dim, output_dim)
tmp = torch.randn(1,8,2)
h0 = torch.randn(layer_dim, tmp.size(0), hidden_dim)
output, hn = rnn(tmp, h0)
output = fc(output)
print(output.shape, hn.shape)

torch.Size([1, 8, 1]) torch.Size([1, 1, 16])
